In [ ]:
conda install -c cyclus java-jdk

In [ ]:
pip install pyspark findspark

In [1]:
import pyspark
import json
import findspark
import pandas as pd
findspark.init()
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
from pyspark.sql import SparkSession, types

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [5]:
df1 = spark.read.json(r"C:\Users\Emre\Downloads\DE_TEST\input_data\category_names.json")
df1.printSchema()
df1.show()

root
 |-- _corrupt_record: string (nullable = true)
 |-- category: string (nullable = true)
 |-- category_id: string (nullable = true)

+---------------+--------------------+-----------+
|_corrupt_record|            category|category_id|
+---------------+--------------------+-----------+
|              [|                null|       null|
|           null|"Other (none of t...|        249|
|           null|           "Billing"|         63|
|           null|           "Handset"|         67|
|           null|         "Your plan"|         72|
|           null| "Customer services"|         71|
|           null|  "Network coverage"|         69|
|           null|"International ro...|         68|
|           null|         "Purchases"|        770|
|           null|"Expectations not...|         70|
|           null|             "Fraud"|         65|
|              ]|                null|       null|
+---------------+--------------------+-----------+



In [6]:
df2 = spark.read.option("multiline","true").json(r"C:\Users\Emre\Downloads\DE_TEST\input_data\complaints.json")
df2.printSchema()
df2.show()

root
 |-- category_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- service_id: string (nullable = true)
 |-- text: string (nullable = true)

+-----------+--------------------+----------+--------------------+
|category_id|          created_at|service_id|                text|
+-----------+--------------------+----------+--------------------+
|        249|"2018-08-21 12:34...|     53014|"I am getting cal...|
|         63|"2018-08-29 14:03...|       409|"I had a mobile a...|
|         63|"2018-08-24 06:11...|     53785|"Charging me doub...|
|        249|"2018-08-23 17:30...|       349|"19 August unfort...|
|         67|"2018-09-05 11:54...|       399|"Hi I have phoned...|
|         72|"2018-09-06 13:06...|       620|"When I originall...|
|         72|"2018-09-04 12:28...|     51870|"I have been aske...|
|         67|"2018-09-06 08:28...|     15708|"I purchased andr...|
|         63|"2018-08-23 03:24...|       218|"I currently have...|
|         63|"2018-09-04 13

In [7]:
df3 = spark.read.json(r"C:\Users\Emre\Downloads\DE_TEST\input_data\service_names.json")
df3.printSchema()
df3.show()

root
 |-- _corrupt_record: string (nullable = true)
 |-- id: string (nullable = true)
 |-- service_name: string (nullable = true)

+---------------+-----+--------------------+
|_corrupt_record|   id|        service_name|
+---------------+-----+--------------------+
|              [| null|                null|
|           null|53014|            "Mobile"|
|           null|  409|            "Mobile"|
|           null|53785|        "Sky Mobile"|
|           null|  349|            "Mobile"|
|           null|  399|"Mobile (Pay Mont...|
|           null|  620|            "Mobile"|
|           null|51870|            "Mobile"|
|           null|15708| "Mobile (Pay & Go)"|
|           null|  218|            "Mobile"|
|           null|  252|            "Mobile"|
|           null|20564|       "Mobile SIMS"|
|           null|20881|"Mobile Phone Ser...|
|           null|  109|            "Mobile"|
|           null| 7482|           "Mobiles"|
|           null|  332|            "Mobile"|
|           nu

In [21]:
df2df1df3 = df2.join(df1).where(df2["category_id"] == df1["category_id"]).drop(df1['category_id'])\
    .join(df3).where(df2["service_id"] == df3["id"]).drop(df3['id'])  \
    .drop(df3['_corrupt_record'])  \
    .drop(df1['_corrupt_record'])  \
    .withColumnRenamed("service_name","Service_Name") \
    .withColumnRenamed("category","Category_Name") \
    .withColumnRenamed("created_at","Complaint_Created_At") \
    .withColumnRenamed("text","Complaint_Text")  \
    .write.csv("Process.csv") \
    .show()

+-----------+--------------------+----------+--------------------+--------------------+--------------------+
|category_id|Complaint_Created_At|service_id|      Complaint_Text|       Category_Name|        Service_Name|
+-----------+--------------------+----------+--------------------+--------------------+--------------------+
|        249|"2018-08-21 12:34...|     53014|"I am getting cal...|"Other (none of t...|            "Mobile"|
|         63|"2018-08-29 14:03...|       409|"I had a mobile a...|           "Billing"|            "Mobile"|
|         63|"2018-08-24 06:11...|     53785|"Charging me doub...|           "Billing"|        "Sky Mobile"|
|        249|"2018-08-23 17:30...|       349|"19 August unfort...|"Other (none of t...|            "Mobile"|
|         67|"2018-09-05 11:54...|       399|"Hi I have phoned...|           "Handset"|"Mobile (Pay Mont...|
|         72|"2018-09-06 13:06...|       620|"When I originall...|         "Your plan"|            "Mobile"|
|         72|"2018-